In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import gcsfs
import dask.dataframe as ddf
plt.rcParams['figure.figsize'] = (15,10)
%matplotlib inline

In [32]:
path = 'gs://pangeo-data/NOAA_buoydata.parquet'

df = ddf.read_parquet(path)
df

,time,id,lat,lon,temp,ve,vn,spd,var_lat,var_lon,var_tmp
npartitions=73,,,,,,,,,,,
,datetime64[ns],int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


In [35]:
df = df[['lon', 'lat', 've', 'vn']].dropna().compute()
df

,lon,lat,ve,vn
1,274.772,-1.242,-49.214,36.778
2,274.657,-1.176,-57.514,28.439
3,274.548,-1.131,-48.625,28.747
4,274.468,-1.064,-50.787,34.399
5,274.351,-0.997,-59.573,34.328
...,...,...,...,...
511995,94.624,-25.758,7.851,-13.752
511996,94.610,-25.789,-7.761,-18.607
511997,94.591,-25.830,-5.754,-17.724
511998,94.585,-25.858,-10.730,-13.769


In [36]:
from xhistogram.core import histogram
import dask.array as dsa

In [37]:
lon = dsa.from_array(df.lon.values, chunks=4_000_000)
lat = dsa.from_array(df.lat.values, chunks=4_000_000)
u = dsa.from_array(df.ve.values, chunks=4_000_000)
v = dsa.from_array(df.vn.values, chunks=4_000_000)

In [38]:
lon

dask.array<array, shape=(35736560,), dtype=float64, chunksize=(4000000,), chunktype=numpy.ndarray>

In [44]:
lon_bins = np.arange(0, 361, 1)
lat_bins = np.arange(-80, 81, 1)

n_obs = histogram(lat, lon, bins=[lat_bins, lon_bins]).compute()
n_obs

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [176, 391, 355, ..., 109, 273, 282],
       [222, 272, 423, ..., 137, 196, 217],
       [ 79,  76, 199, ...,  18,  18,  89]])

In [52]:
lon_c = 0.5*(lon_bins[:-1] + lon_bins[1:])
lat_c = 0.5*(lat_bins[:-1] + lat_bins[1:])

In [46]:
u_binned = histogram(lat, lon, bins=[lat_bins, lon_bins], weights=u).compute()/n_obs
v_binned = histogram(lat, lon, bins=[lat_bins, lon_bins], weights=v).compute()/n_obs
u2_binned = histogram(lat, lon, bins=[lat_bins, lon_bins], weights=u**2).compute()/n_obs
v2_binned = histogram(lat, lon, bins=[lat_bins, lon_bins], weights=v**2).compute()/n_obs

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [47]:
eke_drifters = 0.5 * (u2_binned - u_binned**2 + v2_binned - v_binned**2)

In [57]:
ds = xr.Dataset({'EKE': (('lat', 'lon'), eke_drifters),
                 'n_obs': (('lat', 'lon'), n_obs)},
               coords={'lon': lon_c, 'lat': lat_c, 
                       'lon_bins': lon_bins, 'lat_bins': lat_bins})
ds

<xarray.Dataset>
Dimensions:   (lat: 160, lat_bins: 161, lon: 360, lon_bins: 361)
Coordinates:
  * lon       (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat       (lat) float64 -79.5 -78.5 -77.5 -76.5 ... 76.5 77.5 78.5 79.5
  * lon_bins  (lon_bins) int64 0 1 2 3 4 5 6 7 ... 354 355 356 357 358 359 360
  * lat_bins  (lat_bins) int64 -80 -79 -78 -77 -76 -75 -74 ... 75 76 77 78 79 80
Data variables:
    EKE       (lat, lon) float64 nan nan nan nan nan ... 215.8 273.0 207.0 266.0
    n_obs     (lat, lon) int64 0 0 0 0 0 0 0 0 0 0 ... 17 53 27 64 35 7 18 18 89

In [58]:
ds.to_netcdf('data/EKE_gdp.nc')